In [2]:
import tensorflow as tf
import keras
import pandas as pd


In [3]:
# loading the dataset
data = pd.read_csv("../DATASET/CSV/Real_Combine.csv")
print(data.head(3))

     T    TM   Tm     SLP     H   VV    V    VM      PM 2.5
0  7.4   9.8  4.8  1017.6  93.0  0.5  4.3   9.4  219.720833
1  7.8  12.7  4.4  1018.5  87.0  0.6  4.4  11.1  182.187500
2  6.7  13.4  2.4  1019.4  82.0  0.6  4.8  11.1  154.037500


In [5]:
# pre-process
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [6]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int("num_layers",2,20)):
        model.add(keras.layers.Dense(units=hp.Int("units_"+str(i),
                                                  min_value=32,
                                                  max_value=512,
                                                  step=32),
                                     activation="relu"))
    model.add(keras.layers.Dense(units=1,activation="linear"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate",[1e-2,1e-3,1e-4])),

        loss = "mean_absolute_error",
        metrics = ["mean_absolute_error"]

    )

    return model


In [14]:
import keras_tuner
tuner = keras_tuner.RandomSearch(
    build_model,
    objective="val_mean_absolute_error",
    max_trials = 5,
    executions_per_trial = 3,
    overwrite=True,
    directory = "HP_result",
    project_name = "Air_quality_index"

)

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)



In [15]:
tuner.search(X_train,y_train,epochs=50,validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 37s]
val_mean_absolute_error: 44.42449188232422

Best val_mean_absolute_error So Far: 41.67247009277344
Total elapsed time: 00h 01m 51s
INFO:tensorflow:Oracle triggered exit


In [16]:
tuner.results_summary()

Results summary
Results in HP_result\Air_quality_index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 5
units_0: 224
units_1: 96
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
Score: 41.67247009277344
Trial summary
Hyperparameters:
num_layers: 10
units_0: 384
units_1: 64
learning_rate: 0.01
units_2: 384
units_3: 448
units_4: 320
units_5: 128
units_6: 448
units_7: 224
units_8: 160
units_9: 128
units_10: 480
units_11: 192
units_12: 256
units_13: 192
units_14: 96
units_15: 480
units_16: 32
units_17: 384
Score: 42.14657465616862
Trial summary
Hyperparameters:
num_layers: 9
units_0: 128
units_1: 480
learning_rate: 0.01
units_2: 480
units_3: 160
units_4: 64
units_5: 256
units_6: 352
units_7: 480
units_8: 256
units_9: 224
units_10: 224
units_11: 480
units_12: 512
units_13: 256
units_14: 480
units_15: 288
units_16: 128
units_17: 288
Score: 42.37308247884115
Trial summary
Hyperparameters:
num_layers: 18
units